# Machine Learning Engineer Nanodegree - capstone project

## Predicting earthquakes


Forecasting natural disasters is a critical part of the field of earth science, 
due to the impact of early alarming on decreasing the toll of such catastrophes, 
both with regards to human life, as well as economic loss. Therefore, the goal
of this capstone project was to predict the time to the next earthquake from
data gathered in a laboratory setup.

This project is an entry for a Kaggle competition that can be viewed here:
https://www.kaggle.com/c/LANL-Earthquake-Prediction

In order for this notebook to run, the training and the testing data needs
to be downloaded from this source:
https://www.kaggle.com/c/LANL-Earthquake-Prediction/data

The 'data' folder needs to be in the same folder as this notebook. In the data folder,
there should be a 'train.csv' file that contains all of the training data, 
'sample_submission.csv' file that contains names of the segments of the training data, 
as well as a 'test' folder, that contains all of the test data segments.

## IMPORTANT NOTES:
### ___
## 1. As this project uses a layer optimized for GPU use, it requires running on a computer with a GPU!
## (Only true for the final architecture, i.e. the pure-RNN solution)
### ___
## 2. The values the the training and validation loss area usually higher than the resulting testing error, most likely due to the content of the testing data

In [ ]:
# ------------------------------------------------- IMPORTS -----------------------------------------------
import numpy as np
import pandas as pd 
import os
import math

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

import matplotlib.pyplot as plt


# --------------------------------------------- HELPER FUNCTIONS ------------------------------------------

def plot_model_history(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.plot(loss, 'b', label = "Training mae")
    plt.plot(val_loss, 'r', label = "Validation mae")
    plt.title("Training and validation mae")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    
def plot_data(x, y, time_to_failure, title):
    fig, ax1 = plt.subplots(figsize=(16, 8))
    plt.title(title)
    plt.plot(x, y, color='r')
    ax1.set_ylabel(title, color='r')
    ax1.set_xlabel('Experiment time [s]')
    plt.legend([title], loc=(0.75, 0.9))
    ax2 = ax1.twinx()
    plt.plot(x, time_to_failure, color='g')
    ax2.set_ylabel('Time to failure [s]', color='g')
    plt.legend(['time_to_failure'], loc=(0.75, 0.85))
    plt.grid(False)
    
def get_callbacks(model_name):
    return [ModelCheckpoint(f'{model_name}.hdf5', monitor='val_loss', save_best_only=True, save_weights_only=False, period=3),
            LearningRateScheduler(lambda x: 1. / (1000. * (x + 1)), verbose=1)]

def check_for_gpu():
    # confirm TensorFlow sees the GPU
    from tensorflow.python.client import device_lib
    assert 'GPU' in str(device_lib.list_local_devices()), 'Tensorflow does not see GPU, model will not work!'

    # confirm Keras sees the GPU
    from keras import backend
    assert len(backend.tensorflow_backend._get_available_gpus()) > 0 , 'Keras does not see GPU, model will not work!'
    
    
# ----------------------------------------- DATA PREPROCESSING FUNCTIONS ----------------------------------
def get_preprocessed_raw_data(data):
    
    """
    this function splits the one very long input measurement into many (4194 if the full dataset is used) shorter measurements
    (150000 datapoints each, the same length as each segment of the test dataset)
    In addition, it modifies the dimensionality from 1D to 2D
    I did this after some period of trial and error with Conv1D layers,
    modifying the dimensionality and using Conv2D layers gave much better results
    """

    step = 150000 # size of training data
    columns = 400 # 150000 = 375*400, this number of colums makes the shape of the resulting 2D array as close to square as possible
    sets = math.floor(len(data)/step)

    x_train = np.zeros(shape=(sets, int(step/columns), columns)) 
    y_train = np.zeros(shape=(sets))

    # cut the data into equally-sized chunks
    for i, each in enumerate(range(0, len(data), step)):
        if i == sets:
            break # handling dataset size that does not divide fully into 150 000 long chunks
        print(f'Reshaping set number {i+1}/{sets}', end='\r')
        x_train[i] = np.reshape([e[0] for e in data[each:each+step]], (-1, columns)) # reshape 1D into 2D
        y_train[i] = data[each+step-1][1]

    print()  
    x_train = x_train.reshape(*x_train.shape, 1)
    assert x_train.shape == (sets, int(step/columns), columns, 1)
    
    return x_train, y_train
    
# the product of n_steps and step_length must be equal to the test data segment length (150000)
# global variables, used in two different methods
n_steps = 150
step_length = 1000
assert n_steps*step_length == 150000 , 'Incorrect values of n_steps or step_length'

# reshapes the data into desired step_length
def get_slices(current, n_steps=n_steps, step_length=step_length, last_index=None):
    if last_index == None:
        last_index=len(current)
    temp = (current[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1))
    temp = np.ptp(temp, axis=1) # calculate the peak-to-peak
    return temp.reshape(n_steps, 1)

def get_preprocessed_statistical_data_generators(data, train_validation_split = 0.2):
    """
    This function returns the training and validation generators required by models 3 and 4.
    The data returned is the peak-to-peak value from a slice of size "step_length"
    """
    def get_generator(data, n_steps=n_steps, step_length=step_length, min_index=0, max_index=None, batch_size=16):
        if max_index is None:
            max_index = len(data) - 1

        while True:
            rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
            samples = np.zeros((batch_size, n_steps, 1))
            targets = np.zeros(batch_size, )

            for j, row in enumerate(rows):
                samples[j] = get_slices(data[:, 0], last_index=row)
                targets[j] = data[row - 1, 1]
            yield samples, targets

    batch_size = 64
    validation_set_size = math.floor(len(data) * train_validation_split)

    train_gen = get_generator(data, batch_size=batch_size, min_index=validation_set_size + 1)
    valid_gen = get_generator(data, batch_size=batch_size, max_index=validation_set_size)
    
    return train_gen, valid_gen


# ------------------------------ HELPER FUNCTIONS FOR CREATING SUBMISSION FILES ---------------------------

def save_submission(model_name, raw_data_model = True):
    # Load submission file
    submission = pd.read_csv('./data/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

    # Load each test data, create the feature matrix, get numeric prediction
    for i, seg_id in enumerate(submission.index):
        print(f'Predicting result for segment {i+1}/{len(submission.index)} (seg_id:{seg_id})', end='\r')
        seg = pd.read_csv(f'./data/test/{seg_id}.csv')
        current = seg['acoustic_data'].values
        # different model types require different data modification, hence the if statement below
        if raw_data_model:
            prepared = np.reshape(current, (-1, 400))
            prepared = prepared.reshape(*prepared.shape, 1)
            submission.time_to_failure[i] = model.predict(np.expand_dims(prepared, 0))
        else:
            submission.time_to_failure[i] = model.predict(np.expand_dims(get_slices(current), 0))

    # Save
    submission.to_csv(f'submission-{model_name}.csv')    
    print(submission.head(10))    

In [ ]:
"""
Reading the data. 

Please keep in mind that, for the ease of use, I am reading only the first  1e8 rows (~15%) of the training csv file.
TO ACHIEVE GOOD RESULTS, please comment out the "nrows=1e8" parameter.
But beware, that the training file is has over 9GB!
"""

training_data = pd.read_csv("./data/train.csv", nrows=1e8,
                            dtype={"acoustic_data": np.float32, "time_to_failure": np.float32})

## Visualizing the data

In [ ]:
# Sampling only every 100th of the datapoints for clarity
acoustic_data = training_data['acoustic_data'][::100]
time_to_failure = training_data['time_to_failure'][::100]

# calculating the x axis values - 4e6 datapoints equal one second
measurement_frequency = 4e6
x = [each/measurement_frequency for each in range(int(len(training_data)))][::100]

plot_data(x, acoustic_data, time_to_failure, 'Acoustic data and time to failure')

# very important to delete all that we do not need, due to the sheer size of the training dataset
del acoustic_data
del time_to_failure

In [ ]:
# calculating statistics over the dataset with points_to_average slice sizes
points_to_average = 1000
acoustic_data = np.array(training_data['acoustic_data']).reshape(-1, points_to_average)

# visualizing the statistical function I had the best results with, namely peak-to-peak
statistics = np.ptp(acoustic_data, axis=1)
time_to_failure = training_data['time_to_failure'][::points_to_average]
x = [each/4e6 for each in range(len(training_data))][::points_to_average]

plot_data(x, statistics, time_to_failure, 'Peak-to-peak')


del acoustic_data
del time_to_failure
del x

## Preparing data for feeding to models

In [ ]:
"""
after visualizing, we only need the values
VERY IMPORTANT - if this cell is not run, none of the models will work
If not, there will be a TypeError: unhashable type: 'slice'
"""
if type(training_data) is not np.ndarray: # this cannot be done more than once, hence the check
    training_data = training_data.values

## Model 1 - CNN fed with raw data

In [ ]:
x_train , y_train = get_preprocessed_raw_data(training_data)

In [ ]:
model_name = 'model_v1'
model = Sequential()

input_shape=(None, *x_train.shape[1:])[1:]
filters = 8

model.add(Conv2D(filters=filters, kernel_size=4, strides=2, padding='same', 
    activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=2, strides=1))
model.add(Conv2D(filters=int(2*filters), kernel_size=2, strides=2, padding='same', 
    activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=int(4*filters), kernel_size=2, strides=2, padding='same', 
    activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=int(8*filters), kernel_size=2, strides=2, padding='same', 
    activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation = 'relu'))

model.summary()

In [ ]:
model.compile(optimizer=Nadam(), loss="mae")
callbacks = get_callbacks(model_name)

In [ ]:
model.fit(x_train,
          y_train,
          batch_size=64,
          epochs=20,
          callbacks=callbacks,
          validation_split=0.2,
          verbose=1)

In [ ]:
plot_model_history(model.history)
model.load_weights(f'{model_name}.hdf5')

In [ ]:
"""
This cell only creates the submission file 

Can be completely skipped with no harm
"""

save_submission(model_name, raw_data_model = True)

## Model 2 - CNN + RNN fed with raw data

In [ ]:
try: # no need to recalculate if already defined 
    x_train
    y_train
    print('Re-used x_train and y_train from the previous model')
except NameError:
    x_train , y_train = get_preprocessed_raw_data(training_data)

In [ ]:
model_name = 'model_v2'
model = Sequential()

filters = 32

input_shape=(None, *x_train.shape[1:])[1:]
print(input_shape[:-1])

model.add(Conv2D(filters=filters, kernel_size=8, strides=2, padding='same', 
      activation='relu', input_shape=input_shape, name = '1st'))
model.add(Conv2D(filters=int(filters/2), kernel_size=4, strides=2, padding='same', 
      activation='relu', name = '2nd'))
model.add(Conv2D(filters=int(filters/4), kernel_size=2, strides=2, padding='same', 
      activation='relu', name = '3rd'))
conv_shape = model.get_layer('3rd').output_shape[1:]
print(conv_shape)
model.add(Reshape((conv_shape[0], conv_shape[1] * conv_shape[2])))
model.add(CuDNNGRU(int(filters/8)))
model.add(Dense(int(filters/2), activation = 'relu'))
model.add(Dense(1, activation = 'relu'))

model.summary()

In [ ]:
model.compile(optimizer=Nadam(), loss="mae")
callbacks = get_callbacks('model_v2')

In [ ]:
# WARNING: this model takes quite a long time to train (longest of all 4)
model.fit(x_train,
          y_train,
          batch_size=16,
          epochs=20,
          callbacks=callbacks,
          validation_split=0.2,
          verbose=1)

In [ ]:
plot_model_history(model.history)
model.load_weights(f'{model_name}.hdf5')

In [ ]:
"""
This cell only creates the submission file 

Can be completely skipped with no harm
"""

save_submission(model_name, raw_data_model = True)

## Model 3 - CNN + RNN fed with peak-to-peak data

In [ ]:
"""
This model requires having a GPU
the method below makes sure that both Keras and Tensorflow see the GPU
If this check fails, the model will not work
"""

check_for_gpu()

In [ ]:
# this needs to be re-done even if the last model already used it. If not, will cause a StopIteration error.
train_gen, valid_gen = get_preprocessed_statistical_data_generators(training_data)

In [ ]:
model_name = 'model_v3'

model = Sequential()
cells = 64

input_shape=(None, 1)

model.add(CuDNNGRU(cells, input_shape=input_shape, name = 'rnn'))
rnn_shape = model.get_layer('rnn').output_shape[1:]
model.add(Reshape((*rnn_shape, 1)))
model.add(Conv1D(filters=int(cells/2), kernel_size=8, strides=2, padding='same', 
      activation='relu'))
model.add(Conv1D(filters=int(cells/4), kernel_size=4, strides=2, padding='same', 
      activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dense(int(cells/4), activation = 'relu'))
model.add(Dense(1))

model.summary()

In [ ]:
model.compile(optimizer=Nadam(), loss="mae")
callbacks = get_callbacks(model_name)

In [ ]:
model.fit_generator(train_gen,
                    steps_per_epoch=1000,
                    epochs=10,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=200)

In [ ]:
plot_model_history(model.history)
model.load_weights(f'{model_name}.hdf5')

In [ ]:
"""
This cell only creates the submission file 

Can be completely skipped with no harm
"""

save_submission(model_name, raw_data_model = False)

## Model 4 - pure RNN fed with peak-to-peak data

In [ ]:
"""
This model requires having a GPU
the method below makes sure that both Keras and Tensorflow see the GPU
If this check fails, the model will not work
"""

check_for_gpu()

In [ ]:
train_gen, valid_gen = get_preprocessed_statistical_data_generators(training_data)

In [ ]:
model_name = 'model_v4'
model = Sequential()

model.add(CuDNNGRU(64, input_shape=(None, 1)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.summary()

In [ ]:
model.compile(optimizer=Nadam(), loss="mae")
callbacks = get_callbacks(model_name)

In [ ]:
model.fit_generator(train_gen,
                    steps_per_epoch=1000,
                    epochs=10,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=200)

In [ ]:
plot_model_history(model.history)
model.load_weights(f'{model_name}.hdf5')

In [ ]:
"""
This cell only creates the submission file 

Can be completely skipped with no harm
"""

save_submission(model_name, raw_data_model = False)